<a href="https://colab.research.google.com/github/AlexKressner/Vorlesung-Industrielles-Management/blob/main/MasterPlanningModell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Master Planning

## Modell

### Indexmengen
$l \in L$ : Menge der Lieferanten

$i \in I$ : Menge der Produktionsstandorte

$j \in J$ : Menge der Märkte

$t \in T$ : Menge der Wochen

$p \in P$ : Menge der Produkte

$VP \subseteq P$ : Menge der Vorprodukte

$FP \subseteq P$ : Menge der Fertigprodukte

Es gilt $P=VP \cup FP$ und $VP \cap FP = \emptyset$

$A_p$ : Menge der Fertigprodukte, die Vorprodukt $p \in VP$ enthalten



### Variablen
$Z_{lipt} \ge 0$ : Beschaffungsmenge von Produkt $p$ bei Lieferant $i$ für Werk $i$ in Woche $t$

$X_{ipt} \ge 0$ : Produktionsmenge von Produkt $p$ am Werk $i$ in Woche $t$

$O_{it} \ge 0$ : Nutzung zusätzlicher Kapazitäten im Werk $i$ in Woche $t$

$B_{ipt} \ge 0$ : Bestand von Produkt $p$ im Werk $i$ in Woche $t$

$Y_{ijpt} \ge 0$ : Distributionsmenge von Werk $i$ in Markt $j$ von Produkt $p$ in Woche $t$

### Parameter

$bc_{lp}$ : Kosten für die Beschaffung von Produkt $p$ von Lieferant $l$

$btc_{lip}$ : Kosten für den Transport von Produkt $p$ von Lieferant $l$ zum Werk $i$

$lc_{p}$ : Lagerkosten für Produkt $p$

$pc_{ip}$ : Kosten für die Produktion von Produkt $p$ im Werk $i$

$dtc_{ijp}$ : Kosten für den Transport von Produkt $p$ von Werk $i$ zum Markt $j$

$oc^+_{i}$ : Kosten für Nutzung zusätzlicher Kapazitäten im Werk $i$

$a_{pp'}$ : Direktbedarfskoeffizient Fertigprodukt $p'\in FP$ für Vorprodukt $p \in VP$, d.h wie viele Einheiten von Vorprodukt $p$ benötige ich zur Herstellung von Fertigprodukt $p'$

$bcap_{lpt}$ : Beschaffungskapazitäten von Liefernant $l$ von Produkt $p$ in Woche $t$

$pcap_{it}$ : Produktionskapazitäten von Werk $i$ in Woche $t$

$pcap^+_{it}$ : Zusätzliche Produktionskapazitäten von Werk $i$ in Woche $t$

$d_{jpt}$ : Nachfrage im Markt $j$ in Woche $t$

### Zielfunktion
Min $K = \sum_{l,i,p \in VP,t} bc_{lp} * Z_{lipt} + \sum_{l,i,p \in VP,t} tbc_{lip} * Z_{lipt} + \sum_{i,p \in FP,t} pc_{ip} * X_{ipt} + \sum_{i,p,t} lc_{p} * B_{ipt} + \sum_{i,t} oc^+_{it} * O_{it} + \sum_{i,j,p,t} dtc_{ijp} * Y_{ijpt}$

### Nebenbedingungen

**(1) Lieferantenkapazitäten**

$\sum_{i} Z_{lipt} \le bcap_{lpt}$

$∀ l,p \in VP, t$


**(2) Produktionskapazitäten**

$\sum_{p \in FP} X_{ipt} \le pcap_{it} + O_{it}$

$∀ i, t$

**(3) Beschränkung der Zusatzkapazitäten**

$O_{it} \le pcap^+_{it}$

$∀ i, t$

**(4) Lagerbilanzgleichung für die Fertigprodukte**

$B_{ipt} = B_{ip,t-1} + X_{ipt} - \sum_j Y_{ijpt}$

$∀ i,p \in FP, t$

**(5) Lagerbilanzgleichung für die Vorprodukte**

$B_{ipt} = B_{ip,t-1} + \sum_l Z_{lipt} - \sum_{p' \in A_{p} } a_{pp'} * X_{ip't}$

$∀ i,p \in VP, t$

**(6) Nachfragebefriedigung**

$\sum_i Y_{ijpt} = d_{jpt}$

$∀ j,p \in FP, t$

## Implementierung

In [ ]:
# Notwendigen Programminstallationen
# pip als Paketmanager
!pip install -U -q pip
!pip install -q ortools
# Laden des Programms
from ortools.linear_solver import pywraplp

In [ ]:
# Solver mit SCIP als Backend.
# SCIP implementiert Simplex, Branch-and-Bound, etc
solver = pywraplp.Solver.CreateSolver('SCIP')

## Datenaufbereitung



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Ordner finden
! ls drive/MyDrive/Industrielles_Management/Daten/MasterPlanning

Beschaffungskapazitäten.csv	Produktionskapazitäten.csv
Beschaffungskosten.csv		Produktionskosten.csv
Direktbedarfskoeffizienten.csv	Transportkosten_Beschaffung.csv
Lagerkosten.csv			Transportkosten_Distribution.csv
Nachfrage.csv


In [ ]:
# Pfad zurückgeben
! cd drive/MyDrive/Industrielles_Management/Daten/MasterPlanning && pwd

/content/drive/MyDrive/Industrielles_Management/Daten/MasterPlanning


In [ ]:
# Daten laden
import pandas as pd

In [ ]:
path = "/content/drive/MyDrive/Industrielles_Management/Daten/MasterPlanning"

In [ ]:
# Kapazitäten
beschaffung_kapa = pd.read_csv(f"{path}/Beschaffungskapazitäten.csv", sep=";")
produktion_kapa = pd.read_csv(f"{path}/Produktionskapazitäten.csv", sep=";")

In [ ]:
# Kosten
produktion_kosten = pd.read_csv(f"{path}/Produktionskosten.csv", sep=";", decimal=",")
beschaffung_kosten = pd.read_csv(f"{path}/Beschaffungskosten.csv", sep=";", decimal=",")
lager_kosten = pd.read_csv(f"{path}/Lagerkosten.csv", sep=";", decimal=",")
transport_beschaffung_kosten = pd.read_csv(f"{path}/Transportkosten_Beschaffung.csv", sep=";", decimal=",")
transport_distribution_kosten = pd.read_csv(f"{path}/Transportkosten_Distribution.csv", sep=";", decimal=",")

In [ ]:
# Direktbedarfskoeffizienten
koeffizienten = pd.read_csv(f"{path}/Direktbedarfskoeffizienten.csv", sep=";", decimal=",")

In [ ]:
# Nachfrage
nachfrage = pd.read_csv(f"{path}/Nachfrage.csv", sep=";")

## Indexmengen

In [ ]:
I = produktion_kapa["Werk"].unique().tolist() # Menge der Produktionsstandorte

In [ ]:
J = nachfrage["Markt"].unique().tolist() #Menge der Märkte

In [ ]:
L = beschaffung_kapa["Lieferant"].unique().tolist() # Menge der Lieferanten

In [ ]:
P = lager_kosten["Produkt"].unique().tolist() # Menge der Produkte

In [ ]:
VP = koeffizienten["Vorprodukt"].unique().tolist() # Menge der Vorprodukte

In [ ]:
FP = koeffizienten["Fertigprodukt"].unique().tolist() # Menge der Fertigprodukte

In [ ]:
A = {}
for vp in koeffizienten["Vorprodukt"].unique():
  A[vp] = koeffizienten[koeffizienten["Vorprodukt"]==vp]["Fertigprodukt"].tolist()

In [ ]:
T = beschaffung_kapa["Woche"].unique().tolist() # Menge der Planungsperioden (Wochen)

## Entscheidungsvariablen

In [ ]:
infinity = solver.infinity()

In [ ]:
# Beschaffungsmengen der jeweiligen Vorprodukte (Flasche und Vitaminwasser)
Z={}
for l in L:
  for i in I:
    for p in VP:
      for t in T: 
        Z[l,i,p,t] = solver.NumVar(0.0, infinity, f"{l},{i},{p},{t}")

In [ ]:
# Produktionsmenge Fertigprodukte
X={}
for i in I:
  for t in T:
    for p in FP:
      X[i,p,t] = solver.NumVar(0.0, infinity, f"{i},{p},{t}")

In [ ]:
# Distributionsmenge Fertigprodukte
Y={}
for i in I:
  for j in J:
    for t in T: 
      for p in FP:
        Y[i,j,p,t] = solver.NumVar(0.0, infinity, f"{i},{j},{p},{t}")

In [ ]:
# Bestand von Produkt (Vor- und Fertigprodukt)
B={}
for i in I:
  for p in P:
    for t in T:
      B[i,p,t] = solver.NumVar(0.0, infinity, f"{i},{p},{t}")

In [ ]:
# Nutzung von Zusatzkapazitäten
O={}
for i in I:
    for t in T:
      O[i,t] = solver.NumVar(0.0, infinity, f"{i},{t}")

In [ ]:
print('Anzahl Entscheidungsvariablen =', solver.NumVariables())

Anzahl Entscheidungsvariablen = 360


## Parameter

In [ ]:
# Produktionskosten
pc = produktion_kosten.set_index(["Werk","Produkt"]).to_dict("dict")["Produktionskosten"] # Kosten reguläre Produktion in €/ME
oc = produktion_kosten.set_index(["Werk"]).to_dict("dict")["Kosten_pro_Zusatzkapa"] # Kosten Zusatzkapazitäten in €/Kapazitätseinheit

In [ ]:
# Beschaffungskosten
beschaffung_kosten.set_index(["Lieferant","Produkt"], inplace=True)
bc = beschaffung_kosten.to_dict("dict")["Beschaffungskosten"] # Kosten der Beschaffung eines Produktes in €/ME

In [ ]:
# Transportkosten
transport_beschaffung_kosten.set_index(["Lieferant","Werk","Produkt"], inplace=True)
btc = transport_beschaffung_kosten.to_dict("dict")["Transportkosten"] # Kosten Transport bei Beschaffung in €/ME
transport_distribution_kosten.set_index(["Werk","Markt","Produkt"], inplace=True)
dtc = transport_distribution_kosten.to_dict("dict")["Transportkosten"] # Kosten Transport bei Distribution in €/ME

In [ ]:
# Lagerkosten
lager_kosten.set_index(["Produkt"], inplace=True)
lc = lager_kosten.to_dict("dict")["Lagerkosten"] # Kosten Lagerung in €/Woche/ME

In [ ]:
# Nachfrage
nachfrage.set_index(["Markt","Produkt","Woche"], inplace=True)
d = nachfrage.to_dict("dict")["Nachfragemenge"]

In [ ]:
# Direktbedarfskoeffiziente
koeffizienten.set_index(["Vorprodukt","Fertigprodukt"], inplace=True)
a = koeffizienten.to_dict("dict")["Direktbedarfskoeffizient"]

In [ ]:
# Kapazitäten 
produktion_kapa.set_index(["Werk","Woche"], inplace=True)
pcap = produktion_kapa.to_dict("dict")["Kapaztiät"]
pcap_plus = produktion_kapa.to_dict("dict")["Max_Kapa_durch_Zusatzschichten"]
beschaffung_kapa.set_index(["Lieferant","Produkt","Woche"], inplace=True)
bcap = beschaffung_kapa.to_dict("dict")["Kapazität"]

## Berechnung Lösung

In [ ]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('LÖSUNG:')
    print('Zielfunktionswert (Kosten) =', solver.Objective().Value())
else:
    print('Problem hat keine Lösung')

LÖSUNG:
Zielfunktionswert (Kosten) = 30257.260000000002


In [ ]:
for t in T:
  print(f"Woche: {t}")
  for i in I:
    for p in FP:
      val = round(X[i,p,t].solution_value())
    if val > 0:
      print(f"Werk {i}")
      print(f"Produktion: {val}")
    print(f"Fertigwarenbestand: {round(B[i,p,t].solution_value())}")
    print("\t")
  print("\n")


Woche: 1
Werk Curitiba
Produktion: 82
Fertigwarenbestand: 0
	
Werk Quanzhou
Produktion: 250
Fertigwarenbestand: 42
	
Werk East London
Produktion: 250
Fertigwarenbestand: 0
	


Woche: 2
Werk Curitiba
Produktion: 72
Fertigwarenbestand: 2
	
Werk Quanzhou
Produktion: 253
Fertigwarenbestand: 10
	
Werk East London
Produktion: 235
Fertigwarenbestand: 0
	


Woche: 3
Werk Curitiba
Produktion: 78
Fertigwarenbestand: 0
	
Werk Quanzhou
Produktion: 168
Fertigwarenbestand: 0
	
Werk East London
Produktion: 223
Fertigwarenbestand: 0
	


Woche: 4
Fertigwarenbestand: 0
	
Werk Quanzhou
Produktion: 212
Fertigwarenbestand: 22
	
Werk East London
Produktion: 218
Fertigwarenbestand: 0
	


Woche: 5
Werk Curitiba
Produktion: 123
Fertigwarenbestand: 0
	
Werk Quanzhou
Produktion: 150
Fertigwarenbestand: 0
	
Werk East London
Produktion: 275
Fertigwarenbestand: 0
	


